# Multiple planes to HDF

In [ ]:
from h5rdmtoolbox import x2hdf, tutorial, H5PIV

To convert a multiple planes to a single HDF file (multiplane-file), you must provide the plane folders containing the netCDF4 files.<br>
**For building a multiplane-file, there are a few scenarios to be aware of which will result in different setups of the HDF file structure!**

## Planes with unequal recording frequency

If all planes (in this tutorials two planes) are recorded with **different** measurement frequencies (laser may be triggered leading to slight time differences while average frequency is similar!), then data variables cannot be packed into a **single array** because the time vector is different for each plane. Thus, groups are generated for each plane where each data variable has shape (`nt`, `ny`, `nx`). Here, the number of time steps (`nt`) could also be different for each plane. However, assuming, recording physically/conceptionally belongs to the same "case" it is still reasonable to pack all data in one file. **The time-averaged data is in one data array** having the shape (`nz`, `ny`, `nx`):

In [ ]:
plane0_dir, plane1_dir, plane2_dir = tutorial.PIVview.get_multiplane_directories()

In [ ]:
piv_mplane_uef = x2hdf.PIVMultiPlane([plane0_dir, plane1_dir], time_information=[5, 5.2])  # force different planes by passing different meas. freqs.
piv_mplane_uef.convert()
print(f'conversion time: {piv_mplane_uef.conversion_time}, generated file: {piv_mplane_uef.hdf_filename}')

In [ ]:
with H5PIV(piv_mplane_uef.hdf_filename, 'r') as h5m:
    h5m.dump()

## Planes with equal recording frequency

If all planes (in this tutorials two planes) are recorded with the same measurement frequency, then data variables can be packed into a **single array** with shape (`nz`, `nt`, `ny`, `nx`), which in this tutorial is (2, 5, 110, 127). The time-averaged data has shape (`nz`, `ny`, `nx`)

In [ ]:
piv_mplane_ef = x2hdf.PIVMultiPlane([plane0_dir, plane1_dir], time_information=[5, 5])
piv_mplane_ef.convert()
print(f'conversion time: {piv_mplane_ef.conversion_time}, generated file: {piv_mplane_ef.hdf_filename}')

In [ ]:
with H5PIV(piv_mplane_ef.hdf_filename, 'r') as h5m:
    h5m.dump()

---
## Additional Functionalities of the package

1. computing additional (intermediate) measurement planes
2. combining the intermediate planes with the measured planes to a single new, "virtual" multi-plane file

Final, folder/file structure will look like:

Now let's generate **intermediate** planes by interpolating them.  The following builds 6 equally spaced z-planes. As this includes the outer two (measured) planes, only 4 planes will be interpolated:

In [ ]:
with H5PIV(piv_mplane_ef.hdf_filename, 'r+') as h5m:
    h5m.dump()
    h5m.z[:] = [0, 1]

In [ ]:
ipfile = piv_mplane_ef.compute_intermediate_plane(6)

Note that the original measurement file had two planes. The upper method generated actually not 6 but 4 planes because the outer ones already exist in the measurement file

As the two files are separated, we now create a multi-plane file that has all original and interpolated planes in one file. We call such a case a **virtual case**

In [ ]:
vcase_name = piv_mplane_ef.build_virtual_mplane_file(ipfile)  # pass the just created file name

---
## Converting a PIVview "average dat" file
There are three "average" files in pivview "\*reyn.dat", "\*rms.dat", "\*avg.dat". The latter ("\*avg.dat") **must** be exist for the following method!

In [ ]:
from h5rdmtoolbox.x2hdf.piv2hdf.pivview.plane import build_plane_hdf_from_average_dat_files

In [ ]:
avg_filename = tutorial.PIVview.get_avg_file()
hdf_file = build_plane_hdf_from_average_dat_files(avg_filename, None, None,
                                                  target=str(avg_filename).replace('.dat', '.hdf'))  # reyn and rms file don't exist, so pass None!!!

In [ ]:
with H5PIV(hdf_file, 'r') as h5:
    h5.dump()